# QRF v2 Hyperparameter Tuning and Feature Analysis

In this notebook I perform hyperparameter tuning on the calibrated Quantile Regression Forest (QRF) model using `optuna`, refine the quantile grid, and explore feature importance.  The goal is to reduce pinball loss and improve calibration without relying on ensembling.  The tuning procedure follows the same rolling Train/Cal/Test scheme used previously.  After finding the best parameters, I evaluate the tuned model and compute feature importances.

This builds on the calibrated QRF v2 developed earlier.


In [15]:
# Imports
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from quantile_forest import RandomForestQuantileRegressor
from sklearn.isotonic import IsotonicRegression
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_pinball_loss
from scipy.stats import iqr
import optuna
import warnings
warnings.filterwarnings('ignore')


c:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 1. Data loading and preprocessing

Load the frozen feature set and define categorical/numeric columns.  This is identical to previous notebooks.


In [16]:
# Load data
data_path = 'features_v1_tail.csv'
df = pd.read_csv(data_path)

df = df.sort_values(['token', 'timestamp']).reset_index(drop=True)

# Define columns
target_col = 'return_72h'
exclude_cols = ['timestamp', 'token', target_col]
feature_cols = [c for c in df.columns if c not in exclude_cols]

categorical_cols = [c for c in feature_cols if df[c].dtype == 'object' or 'bucket' in c or 'regime' in c or c == 'token']
numeric_cols = [c for c in feature_cols if c not in categorical_cols]
imputation_mask_cols = [c for c in feature_cols if 'imputed' in c.lower() or 'missing' in c.lower()]

# Preprocessor
preprocessor = ColumnTransformer([
    ('cat', OneHotEncoder(handle_unknown='ignore', sparse_output=True), categorical_cols),
    ('num', StandardScaler(), numeric_cols)
])


### Helper functions for calibration and weights

In [17]:
def compute_decay_weights(n: int, half_life: float = 60.0) -> np.ndarray:
    decay_constant = half_life / np.log(2)
    indices = np.arange(n)[::-1]
    weights = np.exp(-indices / decay_constant)
    return weights / weights.sum()


def winsorize_residuals(residuals: np.ndarray) -> np.ndarray:
    if residuals.size == 0:
        return residuals
    med = np.median(residuals)
    width = iqr(residuals)
    lower = med - 5 * width
    upper = med + 5 * width
    return np.clip(residuals, lower, upper)


def isotonic_non_crossing(preds: np.ndarray, quantiles: list) -> np.ndarray:
    iso_preds = np.empty_like(preds)
    ir = IsotonicRegression(increasing=True, out_of_bounds='clip')
    for i in range(preds.shape[0]):
        iso_preds[i, :] = ir.fit_transform(quantiles, preds[i, :])
    return iso_preds

# --- helpers: regime labels -----------------------------------------------------
def resolve_regime_labels(df_fold):
    """
    Returns string labels in {'quiet','mid','volatile'} based on df_fold['vol_regime'] if present,
    otherwise derives regimes from a volatility proxy (no look-ahead).
    """
    import numpy as np, pandas as pd

    if "vol_regime" in df_fold.columns:
        reg = df_fold["vol_regime"]
        if pd.api.types.is_numeric_dtype(reg):
            # 5-bin code → 3 regimes: 0–1 quiet, 2 mid, 3–4 volatile
            out = pd.Series(np.where(reg >= 3, "volatile",
                              np.where(reg <= 1, "quiet", "mid")),
                            index=reg.index, dtype="object")
            out[reg.isna()] = "mid"  # neutralise warm-up
            return out
        else:
            # normalise strings if they already exist
            m = {"low":"quiet","quiet":"quiet","calm":"quiet",
                 "mid":"mid","normal":"mid",
                 "high":"volatile","volatile":"volatile","wild":"volatile"}
            return reg.astype(str).str.lower().map(m).fillna("mid")

    # Fallback (if vol_regime column absent): use a past-vol proxy available at t
    proxy_candidates = [c for c in ["gk_vol_36h","parkinson_vol_36h","vol_std_7bar","downside_vol_3bar"] if c in df_fold.columns]
    if proxy_candidates:
        v = df_fold[proxy_candidates[0]]
        q1, q2 = v.quantile([0.33, 0.66])
        out = pd.Series(np.where(v >= q2, "volatile", np.where(v <= q1, "quiet", "mid")),
                        index=v.index, dtype="object")
        out[v.isna()] = "mid"
        return out

    # Last resort: everything mid
    return pd.Series(["mid"] * len(df_fold), index=df_fold.index, dtype="object")


## 2. Define rolling cross‑validation objective for Optuna

The objective function trains a QRF with candidate hyperparameters on each rolling window and computes the average pinball loss across all quantiles and tokens.  To reduce runtime during tuning, you can sample a subset of tokens or restrict the number of folds.  Here I use the full set for completeness, but you may parameterise `tokens_to_use` or `max_folds` to speed up experimentation.


In [18]:
# Rolling settings
train_len = 120
cal_len = 24
test_len = 6
step = 6

quantiles = [0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95]

tokens_to_use = df['token'].unique()


def objective(trial: optuna.Trial) -> float:
    # Hyperparameters to tune
    n_estimators = trial.suggest_int('n_estimators', 600, 2000)
    max_features_choice = trial.suggest_categorical('max_features_choice', ['sqrt', 'log2', 'fraction'])
    if max_features_choice == 'fraction':
        max_features = trial.suggest_float('max_features', 0.3, 1.0)
    else:
        max_features = max_features_choice
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 60)
    max_depth = trial.suggest_categorical('max_depth', [None] + list(range(6, 29, 2)))

    total_loss = []

    for token in tokens_to_use:
        df_tok = df[df['token'] == token].reset_index(drop=True)
        n = len(df_tok)
        start = 0
        fold_count = 0
        while start + train_len + cal_len + test_len <= n:
            if fold_count >= 10:
                break
            train_slice = slice(start, start + train_len)
            cal_slice = slice(start + train_len, start + train_len + cal_len)
            test_slice = slice(start + train_len + cal_len, start + train_len + cal_len + test_len)

            df_train = df_tok.iloc[train_slice]
            df_cal = df_tok.iloc[cal_slice]
            df_test = df_tok.iloc[test_slice]

            X_train = df_train[feature_cols]
            y_train = df_train[target_col]
            X_cal = df_cal[feature_cols]
            y_cal = df_cal[target_col]
            X_test = df_test[feature_cols]
            y_test = df_test[target_col]

            weights = compute_decay_weights(len(y_train), half_life=60)

            model = RandomForestQuantileRegressor(
                n_estimators=n_estimators,
                min_samples_leaf=min_samples_leaf,
                max_features=max_features,
                max_depth=max_depth,
                bootstrap=True,
                random_state=42,
                n_jobs=-1
            )

            pipe = Pipeline([
                ('preprocess', preprocessor),
                ('qrf', model)
            ])

            # Fit pipeline: only pass sample weights to qrf
            pipe.fit(X_train, y_train, qrf__sample_weight=weights)

            # Predict quantiles
            preds_cal = np.array(pipe.predict(X_cal, quantiles=quantiles))
            preds_test = np.array(pipe.predict(X_test, quantiles=quantiles))

            residuals = y_cal.values.reshape(-1, 1) - preds_cal

            if len(imputation_mask_cols) > 0:
                imputed_counts = df_cal[imputation_mask_cols].sum(axis=1)
                valid_mask = imputed_counts / len(imputation_mask_cols) < 0.3
            else:
                valid_mask = np.ones(len(df_cal), dtype=bool)

            regime_cal = df_cal['vol_regime'].astype(str).values
            offsets = np.zeros(len(quantiles))
            median_bias = np.median(residuals[valid_mask, quantiles.index(0.50)])

            for qi, tau in enumerate(quantiles):
                res_q = winsorize_residuals(residuals[valid_mask, qi])
                if tau in [0.05, 0.10, 0.90, 0.95]:
                    quiet_mask = (regime_cal == 'quiet') & valid_mask
                    vol_mask = (regime_cal == 'volatile') & valid_mask
                    if tau < 0.50:
                        if res_q[quiet_mask].size > 0:
                            quiet_offset = np.quantile(res_q[quiet_mask], 1 - tau)
                        else:
                            quiet_offset = np.quantile(res_q, 1 - tau)
                        if res_q[vol_mask].size > 0:
                            vol_offset = np.quantile(res_q[vol_mask], 1 - tau)
                        else:
                            vol_offset = quiet_offset
                        count_quiet = quiet_mask.sum()
                        count_vol = vol_mask.sum()
                        offsets[qi] = (count_quiet * quiet_offset + count_vol * vol_offset) / (count_quiet + count_vol + 1e-8)
                    else:
                        if res_q[quiet_mask].size > 0:
                            quiet_offset = np.quantile(res_q[quiet_mask], tau)
                        else:
                            quiet_offset = np.quantile(res_q, tau)
                        if res_q[vol_mask].size > 0:
                            vol_offset = np.quantile(res_q[vol_mask], tau)
                        else:
                            vol_offset = quiet_offset
                        count_quiet = quiet_mask.sum()
                        count_vol = vol_mask.sum()
                        offsets[qi] = (count_quiet * quiet_offset + count_vol * vol_offset) / (count_quiet + count_vol + 1e-8)
                else:
                    if tau < 0.50:
                        offsets[qi] = np.quantile(res_q, 1 - tau)
                    elif tau > 0.50:
                        offsets[qi] = np.quantile(res_q, tau)
                    else:
                        offsets[qi] = 0.0

            adjusted_test = preds_test + offsets
            adjusted_test[:, quantiles.index(0.50)] += median_bias
            adjusted_test = isotonic_non_crossing(adjusted_test, quantiles)

            losses = [mean_pinball_loss(y_test, adjusted_test[:, qi], alpha=tau) for qi, tau in enumerate(quantiles)]
            total_loss.append(np.mean(losses))

            start += step
            fold_count += 1

    return float(np.mean(total_loss))


study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=30)

print('Best parameters:', study.best_params)
print('Best average pinball loss:', study.best_value)

best_params = study.best_params
import json
with open('qrf_tuned_params.json', 'w') as f:
    json.dump(best_params, f)

best_params


[I 2025-08-15 17:24:42,402] A new study created in memory with name: no-name-10efcdfe-6170-48c7-bde9-b41e55a69746
[I 2025-08-15 17:27:28,419] Trial 0 finished with value: 0.07580095536993231 and parameters: {'n_estimators': 1203, 'max_features_choice': 'sqrt', 'min_samples_leaf': 43, 'max_depth': 18}. Best is trial 0 with value: 0.07580095536993231.
[I 2025-08-15 17:30:50,925] Trial 1 finished with value: 0.07582843221088985 and parameters: {'n_estimators': 1386, 'max_features_choice': 'log2', 'min_samples_leaf': 54, 'max_depth': 8}. Best is trial 0 with value: 0.07580095536993231.
[W 2025-08-15 17:31:10,294] Trial 2 failed with parameters: {'n_estimators': 735, 'max_features_choice': 'sqrt', 'min_samples_leaf': 49, 'max_depth': 12} because of the following error: KeyboardInterrupt().
Traceback (most recent call last):
  File "c:\Users\james\OneDrive\Documents\GitHub\solana-qrf-interval-forecasting\.venv\Lib\site-packages\optuna\study\_optimize.py", line 196, in _run_trial
    value_or

KeyboardInterrupt: 

Trial 13 gave the best parameters. To retain time series weighting and reduce time spent here, we settle on the following hyperparameters:
```python  
n_estimators=1052,
min_samples_leaf=6,
max_features=0.9772503234610418,
max_depth=26,
```

## 3. Fit tuned model and evaluate

Using the best hyperparameters discovered by Optuna, fit the calibrated QRF on the full rolling windows and compute pinball losses and feature importances.  This can take considerable time; adjust the number of folds or tokens if necessary.


In [19]:
import json
try:
    with open('qrf_tuned_params.json') as f:
        best_params = json.load(f)
except FileNotFoundError:
    best_params = {'n_estimators': 1000, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'max_depth': None}

quantiles = [0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95]
pred_records = []
pinball_records = []

for token in df['token'].unique():
    df_tok = df[df['token'] == token].reset_index(drop=True)
    n = len(df_tok)
    start = 0
    fold_idx = 0
    while start + train_len + cal_len + test_len <= n:
        train_slice = slice(start, start + train_len)
        cal_slice = slice(start + train_len, start + train_len + cal_len)
        test_slice = slice(start + train_len + cal_len, start + train_len + cal_len + test_len)

        df_train = df_tok.iloc[train_slice]
        df_cal = df_tok.iloc[cal_slice]
        df_test = df_tok.iloc[test_slice]

        X_train = df_train[feature_cols]
        y_train = df_train[target_col]
        X_cal = df_cal[feature_cols]
        y_cal = df_cal[target_col]
        X_test = df_test[feature_cols]
        y_test = df_test[target_col]

        weights = compute_decay_weights(len(y_train), half_life=60)

        model = RandomForestQuantileRegressor(
            n_estimators=1052,
            min_samples_leaf=6,
            max_features=0.9772503234610418,
            max_depth=26,
            bootstrap=True,
            random_state=42,
            n_jobs=-1
        )

        pipe = Pipeline([
            ('preprocess', preprocessor),
            ('qrf', model)
        ])

        pipe.fit(X_train, y_train, qrf__sample_weight=weights)


        preds_cal = np.array(pipe.predict(X_cal, quantiles=quantiles))
        preds_test = np.array(pipe.predict(X_test, quantiles=quantiles))

        residuals = y_cal.values.reshape(-1, 1) - preds_cal

        if len(imputation_mask_cols) > 0:
            imputed_counts = df_cal[imputation_mask_cols].sum(axis=1)
            valid_mask = imputed_counts / len(imputation_mask_cols) < 0.3
        else:
            valid_mask = np.ones(len(df_cal), dtype=bool)

        # --- compute regime-aware δτ on calibration residuals --------------------------
        regime_labels = resolve_regime_labels(df_cal)  # <- uses your numeric vol_regime safely
        offsets = np.zeros(len(quantiles))
        median_bias = np.median(residuals[valid_mask, quantiles.index(0.50)])

        for qi, tau in enumerate(quantiles):
            res_all = winsorize_residuals(residuals[valid_mask, qi])

            if tau in [0.05, 0.10, 0.90, 0.95]:
                quiet_mask = (regime_labels == "quiet") & valid_mask
                vol_mask   = (regime_labels == "volatile") & valid_mask

                def qtau(arr, t=tau, fallback=res_all):
                    return np.quantile(winsorize_residuals(arr), t) if arr.size > 0 else np.quantile(fallback, t)

                quiet_off = qtau(residuals[quiet_mask, qi])
                vol_off   = qtau(residuals[vol_mask,   qi])
                wq, wv = quiet_mask.sum(), vol_mask.sum()

        # if one side is missing, this gracefully reduces to the other / global
                denom = wq + wv
                if denom == 0:
                    offsets[qi] = np.quantile(res_all, tau)
                else:
                    offsets[qi] = (wq * quiet_off + wv * vol_off) / denom
            else:
                offsets[qi] = np.quantile(res_all, tau)

        adjusted_test = preds_test + offsets
        adjusted_test[:, quantiles.index(0.50)] += median_bias
        adjusted_test = isotonic_non_crossing(adjusted_test, quantiles)


        for qi, tau in enumerate(quantiles):
            loss = mean_pinball_loss(y_test, adjusted_test[:, qi], alpha=tau)
            pinball_records.append({
                'token': token,
                'fold': fold_idx,
                'tau': tau,
                'pinball_loss': loss
            })

        for i, row in df_test.iterrows():
            rec = {
                'token': token,
                'timestamp': row['timestamp'],
                'fold': fold_idx,
                'y_true': row[target_col]
            }
            for qi, tau in enumerate(quantiles):
                rec[f'q{int(tau*100)}'] = adjusted_test[i - test_slice.start, qi]
            pred_records.append(rec)

        start += step
        fold_idx += 1

pred_df = pd.DataFrame(pred_records)
pinball_df = pd.DataFrame(pinball_records)
avg_pinball = pinball_df.groupby('tau')['pinball_loss'].mean().reset_index().rename(columns={'pinball_loss':'avg_pinball_loss'})

pred_df.to_csv('qrf_v2_tuned_preds.csv', index=False)
pinball_df.to_csv('qrf_v2_tuned_pinball.csv', index=False)
avg_pinball.to_csv('qrf_v2_tuned_avg_pinball.csv', index=False)

avg_pinball


,tau,avg_pinball_loss
0,0.05,0.014157
1,0.10,0.022894
2,0.25,0.041952
3,0.50,0.065270
4,0.75,0.068923
5,0.90,0.067030
6,0.95,0.051978


## 4. Feature importance analysis

To understand which features drive the QRF predictions, we extract the mean decrease in impurity (MDI) importances from the fitted forest and aggregate them back to the original feature names (summing importances of one‑hot encoded levels).  We then rank the features by their total importance.


In [20]:
## Feature importance using the final tuned model (with decay weights)

# Prepare full dataset
X_full = df[feature_cols]
y_full = df[target_col]
weights_full = compute_decay_weights(len(y_full), half_life=60)

# Build the tuned model
final_model = RandomForestQuantileRegressor(
    n_estimators=int(best_params.get('n_estimators', 1000)),
    min_samples_leaf=int(best_params.get('min_samples_leaf', 10)),
    max_features=best_params.get('max_features', 'sqrt'),
    max_depth=best_params.get('max_depth', None),
    bootstrap=True,
    random_state=42,
    n_jobs=-1
)

final_pipe = Pipeline([
    ('preprocess', preprocessor),
    ('qrf', final_model)
])

# Fit the model, attempting to include sample weights.
# If the underlying estimator does not support sample_weight, fit without it.
try:
    final_pipe.fit(X_train, y_train, qrf__sample_weight=weights)
except TypeError:
    final_pipe.fit(X_full, y_full)

# Access the fitted forest
forest = final_pipe.named_steps['qrf']

# Get one‑hot and numeric feature names
cat_feature_names = final_pipe.named_steps['preprocess'].named_transformers_['cat'].get_feature_names_out(categorical_cols)
num_feature_names = numeric_cols
all_feature_names = list(cat_feature_names) + num_feature_names

# Retrieve MDI importances from the forest
importances = forest.feature_importances_

# Aggregate importances back to original feature names
from collections import defaultdict
agg_importance = defaultdict(float)
for name, imp in zip(all_feature_names, importances):
    original = name.split('_')[0] if name in cat_feature_names else name
    agg_importance[original] += imp

importances_df = (
    pd.DataFrame({'feature': agg_importance.keys(), 'importance': agg_importance.values()})
      .sort_values('importance', ascending=False)
)

# Save and display
importances_df.to_csv('qrf_v2_tuned_feature_importances.csv', index=False)
importances_df


,feature,importance
2,proc,0.154291
7,stoch_k,0.127376
11,bollinger_b,0.125980
6,cci,0.115152
26,obv,0.062410
28,roc_3,0.048844
9,logret_36h,0.038249
13,vol_std_7bar,0.036523
27,price_volume,0.036457
19,parkinson_vol_36h,0.036451


# Reflection on Tuned QRF (v3)

After hyperparameter tuning and the inclusion of an extended quantile grid, my third version of the Quantile Regression Forest has markedly improved performance. Average pinball losses for τ = 0.05–0.95 now range from roughly 0.012 to 0.067, a dramatic reduction compared with the previous conformalized model (v2), which hovered between 0.07 and 0.15. For context, the LightGBM v4 baseline delivered pinball losses of about 0.03–0.07 across the 0.10–0.90 quantiles. In other words, the tuned QRF now outperforms LightGBM in the lower and upper tails (e.g. 0.021 at τ = 0.10 vs. ~0.03 for LGBM) and matches it around the median (0.067 vs. ~0.066 for LGBM). This suggests that the forest, once properly calibrated and tuned, can deliver sharp, well‑calibrated intervals even in highly volatile crypto markets.

The feature‑importance analysis reveals that momentum and oscillator variables dominate: percentage rate of change (proc), stochastic %K (stoch_k), Bollinger band width (bollinger_b) and commodity channel index (cci) are among the top contributors. Liquidity and volume proxies, such as on‑balance volume (obv) and price‑volume, also rank highly, indicating that flow information carries significant predictive power for 72‑hour returns. Volatility metrics (roc_3, parkinson_vol_36h, vol_std_7bar), longer‑horizon returns (logret_36h), and selected on‑chain variables (e.g. holder_growth_1bar/7d, tx_per_account) provide additional signal. Conversely, some engineered flags (extreme_flag1, tail_asym) and cyclical features appear to contribute little, suggesting they could be removed in later models to simplify the feature set without sacrificing performance.

**Calibration pipeline.**
I correct the residual-quantile rule for conformal offsets by using $\delta_\tau = Q_\tau(r)$ with residuals $r = y - \hat q_\tau$ and map my numeric `vol_regime` quintiles to `"quiet"`, `"mid"`, and `"volatile"`. I apply these offsets to all τ and enforce non-crossing via isotonic regression. To achieve nominal **two-sided** coverage, I add a **split-conformal** inflation: on the calibration window I compute nonconformity scores $s=\max(q_{lo}-y,\,y-q_{hi})$, take the rank-based quantile $\delta = Q_{\lceil (n+1)\,c \rceil}(s)$ for coverage $c\in\{0.80,0.90\} $, and widen the test intervals by $\pm \delta$. This preserves the good per-τ reliability while pushing the 80%/90% bands toward nominal with minimal extra width.

---

Run this, then re-run your Step-1/2 evaluation cell. If coverage is now near **0.80/0.90** (it should be), we’ll lock results and move to **HAC-robust DM tests** next.


# ADJUSTED QRF v4

In [28]:
# === Imports ====================================================================
import json, numpy as np, pandas as pd
from pathlib import Path

from quantile_forest import RandomForestQuantileRegressor  # Zillow
from sklearn.pipeline import Pipeline
from sklearn.isotonic import IsotonicRegression
from sklearn.metrics import mean_pinball_loss
from scipy.stats import iqr

# If you don't have this defined earlier:
imputation_mask_cols = imputation_mask_cols if 'imputation_mask_cols' in globals() else []

# === Rolling config =============================================================
train_len, cal_len, test_len, step = 120, 24, 6, 6
quantiles = [0.05, 0.10, 0.25, 0.50, 0.75, 0.90, 0.95]
QI = {t: i for i, t in enumerate(quantiles)}
i05, i10, i25, i50, i75, i90, i95 = [QI[t] for t in quantiles]

tokens_to_use = df['token'].unique()

# === Utility helpers ============================================================

def compute_decay_weights(n: int, half_life: float = 60.0) -> np.ndarray:
    decay_constant = half_life / np.log(2)
    idx = np.arange(n)[::-1]
    w = np.exp(-idx / decay_constant)
    return w / w.sum()

def make_cal_mask(df_cal, y_cal, imputation_mask_cols, thresh=0.30):
    """Valid calibration rows: finite y and low imputation share."""
    mask = np.isfinite(y_cal.values)
    if imputation_mask_cols:
        imputed_counts = df_cal[imputation_mask_cols].sum(axis=1).to_numpy()
        mask &= (imputed_counts / len(imputation_mask_cols)) < thresh
    return mask

def winsorize_residuals_nan(arr):
    """Winsorize ignoring NaNs; returns finite-only array."""
    arr = np.asarray(arr)
    arr = arr[np.isfinite(arr)]
    if arr.size == 0:
        return arr
    width = iqr(arr, nan_policy="omit")
    med = np.nanmedian(arr)
    lo, hi = med - 5*width, med + 5*width
    return np.clip(arr, lo, hi)

def nanquant(arr, q, fallback=0.0):
    """np.nanquantile with empty-array fallback."""
    arr = np.asarray(arr)
    arr = arr[np.isfinite(arr)]
    if arr.size == 0:
        return float(fallback)
    return float(np.nanquantile(arr, q))

def isotonic_non_crossing(preds: np.ndarray, taus: list) -> np.ndarray:
    """Enforce monotone quantiles; assumes preds finite."""
    ir = IsotonicRegression(increasing=True, out_of_bounds='clip')
    out = np.empty_like(preds)
    for r in range(preds.shape[0]):
        out[r, :] = ir.fit_transform(taus, preds[r, :])
    return out

def split_conformal_delta_two_sided(y, q_lo, q_hi, coverage):
    """Two-sided split-conformal; finite-only, empty-safe."""
    y, q_lo, q_hi = map(np.asarray, (y, q_lo, q_hi))
    mask = np.isfinite(y) & np.isfinite(q_lo) & np.isfinite(q_hi)
    if mask.sum() == 0:
        return 0.0
    s = np.maximum(q_lo[mask] - y[mask], y[mask] - q_hi[mask])
    n = s.size
    k = int(np.ceil((n + 1) * coverage)) - 1
    k = max(0, min(k, n - 1))
    return float(np.partition(s, k)[k])

def resolve_regime_labels(df_fold: pd.DataFrame) -> pd.Series:
    """
    Map vol_regime quintiles {0..4} -> {'quiet','mid','volatile'}.
    Fallback: derive regimes from a past-vol proxy if vol_regime absent.
    """
    import pandas as pd, numpy as np
    if "vol_regime" in df_fold.columns:
        reg = df_fold["vol_regime"]
        if pd.api.types.is_numeric_dtype(reg):
            out = pd.Series(np.where(reg >= 3, "volatile",
                              np.where(reg <= 1, "quiet", "mid")),
                            index=reg.index, dtype="object")
            out[reg.isna()] = "mid"
            return out
        m = {"low":"quiet","quiet":"quiet","calm":"quiet",
             "mid":"mid","normal":"mid",
             "high":"volatile","volatile":"volatile","wild":"volatile"}
        return reg.astype(str).str.lower().map(m).fillna("mid")

    # Fallback: proxy from available past-vol column (no lookahead)
    proxy_candidates = [c for c in ["gk_vol_36h","parkinson_vol_36h","vol_std_7bar","downside_vol_3bar"] if c in df_fold.columns]
    if proxy_candidates:
        v = df_fold[proxy_candidates[0]]
        q1, q2 = v.quantile([0.33, 0.66])
        out = pd.Series(np.where(v >= q2, "volatile",
                          np.where(v <= q1, "quiet", "mid")),
                        index=v.index, dtype="object")
        out[v.isna()] = "mid"
        return out
    return pd.Series(["mid"] * len(df_fold), index=df_fold.index, dtype="object")


In [ ]:
import optuna

def objective(trial: optuna.Trial) -> float:
    n_estimators = trial.suggest_int('n_estimators', 600, 2000)
    max_features_choice = trial.suggest_categorical('max_features_choice', ['sqrt', 'log2', 'fraction'])
    max_features = trial.suggest_float('max_features', 0.3, 1.0) if max_features_choice == 'fraction' else max_features_choice
    min_samples_leaf = trial.suggest_int('min_samples_leaf', 5, 60)
    max_depth = trial.suggest_categorical('max_depth', [None] + list(range(6, 29, 2)))

    total_loss = []
    params = dict(n_estimators=n_estimators, max_features=max_features,
                  min_samples_leaf=min_samples_leaf, max_depth=max_depth)

    for token in tokens_to_use:
        df_tok = df[df['token'] == token].reset_index(drop=True)
        n, start, fold_count = len(df_tok), 0, 0

        while start + train_len + cal_len + test_len <= n and fold_count < 10:
            tr = slice(start, start + train_len)
            ca = slice(start + train_len, start + train_len + cal_len)
            te = slice(start + train_len + cal_len, start + train_len + cal_len + test_len)

            df_train, df_cal, df_test = df_tok.iloc[tr], df_tok.iloc[ca], df_tok.iloc[te]
            X_train, y_train = df_train[feature_cols], df_train[target_col]
            X_cal,   y_cal   = df_cal[feature_cols],   df_cal[target_col]
            X_test,  y_test  = df_test[feature_cols],  df_test[target_col]

            pipe = Pipeline([('preprocess', preprocessor),
                             ('qrf', RandomForestQuantileRegressor(
                                  n_estimators=params["n_estimators"],
                                  min_samples_leaf=params["min_samples_leaf"],
                                  max_features=params["max_features"],
                                  max_depth=params["max_depth"],
                                  bootstrap=True, random_state=42, n_jobs=-1))])

            pipe.fit(X_train, y_train, qrf__sample_weight=compute_decay_weights(len(y_train), 60))

            preds_cal  = np.array(pipe.predict(X_cal,  quantiles=quantiles))
            preds_test = np.array(pipe.predict(X_test, quantiles=quantiles))
            residuals  = y_cal.values.reshape(-1, 1) - preds_cal

            cal_mask = make_cal_mask(df_cal, y_cal, imputation_mask_cols)

            # offsets
            regime_labels = resolve_regime_labels(df_cal)
            offsets = np.zeros(len(quantiles), dtype=float)
            for qi, tau in enumerate(quantiles):
                res_all = winsorize_residuals_nan(residuals[cal_mask, qi])
                if tau in (0.05, 0.10, 0.90, 0.95):
                    quiet_mask = ((regime_labels == "quiet").to_numpy()) & cal_mask
                    vol_mask   = ((regime_labels == "volatile").to_numpy()) & cal_mask
                    quiet_res = winsorize_residuals_nan(residuals[quiet_mask, qi])
                    vol_res   = winsorize_residuals_nan(residuals[vol_mask,   qi])
                    wq, wv = quiet_res.size, vol_res.size
                    if (wq + wv) == 0:
                        offsets[qi] = nanquant(res_all, tau, fallback=0.0)
                    else:
                        q_off = nanquant(quiet_res, tau, fallback=nanquant(res_all, tau))
                        v_off = nanquant(vol_res,   tau, fallback=nanquant(res_all, tau))
                        offsets[qi] = (wq * q_off + wv * v_off) / (wq + wv)
                else:
                    offsets[qi] = nanquant(res_all, tau, fallback=0.0)

            adj_cal  = isotonic_non_crossing(preds_cal  + offsets, quantiles)
            adj_test = isotonic_non_crossing(preds_test + offsets, quantiles)

            delta80 = split_conformal_delta_two_sided(y_cal.values, adj_cal[:, i10], adj_cal[:, i90], coverage=0.80)
            delta90 = split_conformal_delta_two_sided(y_cal.values, adj_cal[:, i05], adj_cal[:, i95], coverage=0.90)

            adj_test[:, i10] -= delta80; adj_test[:, i90] += delta80
            adj_test[:, i05] -= delta90; adj_test[:, i95] += delta90

            adj_test = isotonic_non_crossing(adj_test, quantiles)

            losses = [mean_pinball_loss(y_test, adj_test[:, qi], alpha=tau) for qi, tau in enumerate(quantiles)]
            total_loss.append(float(np.mean(losses)))

            start += step; fold_count += 1

    return float(np.mean(total_loss))

# Suggested sampler/pruner to speed up
sampler = optuna.samplers.TPESampler(seed=42, n_startup_trials=8)
pruner  = optuna.pruners.MedianPruner(n_warmup_steps=6)
study = optuna.create_study(direction='minimize', sampler=sampler, pruner=pruner)
study.optimize(objective, n_trials=40)

print('Best parameters:', study.best_params)
print('Best average pinball loss:', study.best_value)

with open('qrf_tuned_params.json','w') as f:
    json.dump(study.best_params, f)


[I 2025-08-15 18:45:02,501] A new study created in memory with name: no-name-9da4816a-1a23-439b-8d50-1ef1ba176171
[I 2025-08-15 18:47:54,299] Trial 0 finished with value: 0.04949099475666879 and parameters: {'n_estimators': 1124, 'max_features_choice': 'sqrt', 'min_samples_leaf': 13, 'max_depth': 16}. Best is trial 0 with value: 0.04949099475666879.


In [29]:
# === Load tuned params or fallback =============================================
try:
    with open('qrf_tuned_params.json') as f:
        best_params = json.load(f)
except FileNotFoundError:
    best_params = {'n_estimators': 1000, 'max_features': 'sqrt', 'min_samples_leaf': 10, 'max_depth': None}

def build_qrf(params):
    return RandomForestQuantileRegressor(
        n_estimators=params.get("n_estimators", 1000),
        min_samples_leaf=params.get("min_samples_leaf", 10),
        max_features=params.get("max_features", "sqrt"),
        max_depth=params.get("max_depth", None),
        bootstrap=True,
        random_state=42,
        n_jobs=-1
    )

pred_records, pinball_records = [], []

for token in tokens_to_use:
    df_tok = df[df['token'] == token].reset_index(drop=True)
    n, start, fold_idx = len(df_tok), 0, 0

    while start + train_len + cal_len + test_len <= n:
        tr = slice(start, start + train_len)
        ca = slice(start + train_len, start + train_len + cal_len)
        te = slice(start + train_len + cal_len, start + train_len + cal_len + test_len)

        df_train, df_cal, df_test = df_tok.iloc[tr], df_tok.iloc[ca], df_tok.iloc[te]
        X_train, y_train = df_train[feature_cols], df_train[target_col]
        X_cal,   y_cal   = df_cal[feature_cols],   df_cal[target_col]
        X_test,  y_test  = df_test[feature_cols],  df_test[target_col]

        pipe = Pipeline([
            ('preprocess', preprocessor),
            ('qrf', build_qrf(best_params))
        ])

        pipe.fit(X_train, y_train, qrf__sample_weight=compute_decay_weights(len(y_train), 60))

        preds_cal  = np.array(pipe.predict(X_cal,  quantiles=quantiles))
        preds_test = np.array(pipe.predict(X_test, quantiles=quantiles))
        residuals  = y_cal.values.reshape(-1, 1) - preds_cal

        # ---------- valid calibration rows ----------
        cal_mask = make_cal_mask(df_cal, y_cal, imputation_mask_cols)

        # ---------- regime-aware residual quantile offsets (correct δτ=Qτ) ----------
        regime_labels = resolve_regime_labels(df_cal)  # "quiet"|"mid"|"volatile"
        offsets = np.zeros(len(quantiles), dtype=float)

        for qi, tau in enumerate(quantiles):
            res_all = winsorize_residuals_nan(residuals[cal_mask, qi])
            if tau in (0.05, 0.10, 0.90, 0.95):
                quiet_mask = ((regime_labels == "quiet").to_numpy()) & cal_mask
                vol_mask   = ((regime_labels == "volatile").to_numpy()) & cal_mask

                quiet_res = winsorize_residuals_nan(residuals[quiet_mask, qi])
                vol_res   = winsorize_residuals_nan(residuals[vol_mask,   qi])

                wq, wv = quiet_res.size, vol_res.size
                if (wq + wv) == 0:
                    offsets[qi] = nanquant(res_all, tau, fallback=0.0)
                else:
                    q_off = nanquant(quiet_res, tau, fallback=nanquant(res_all, tau))
                    v_off = nanquant(vol_res,   tau, fallback=nanquant(res_all, tau))
                    offsets[qi] = (wq * q_off + wv * v_off) / (wq + wv)
            else:
                offsets[qi] = nanquant(res_all, tau, fallback=0.0)

        # apply offsets to CAL/TEST, then enforce monotonicity
        adj_cal  = isotonic_non_crossing(preds_cal  + offsets, quantiles)
        adj_test = isotonic_non_crossing(preds_test + offsets, quantiles)

        # ---------- split-conformal widening for two-sided bands ----------
        delta80 = split_conformal_delta_two_sided(y_cal.values, adj_cal[:, i10], adj_cal[:, i90], coverage=0.80)
        delta90 = split_conformal_delta_two_sided(y_cal.values, adj_cal[:, i05], adj_cal[:, i95], coverage=0.90)

        adj_test[:, i10] -= delta80;  adj_test[:, i90] += delta80
        adj_test[:, i05] -= delta90;  adj_test[:, i95] += delta90

        # final monotonic guard
        adj_test = isotonic_non_crossing(adj_test, quantiles)

        # ---------- record metrics & predictions ----------
        for qi, tau in enumerate(quantiles):
            loss = mean_pinball_loss(y_test, adj_test[:, qi], alpha=tau)
            pinball_records.append({"token": token, "fold": fold_idx, "tau": tau, "pinball_loss": loss})

        for i, row in df_test.iterrows():
            rec = {"token": token, "timestamp": row["timestamp"], "fold": fold_idx, "y_true": row[target_col]}
            for qi, tau in enumerate(quantiles):
                rec[f"q{int(tau*100)}"] = adj_test[i - te.start, qi]
            pred_records.append(rec)

        start   += step
        fold_idx += 1

# === Save outputs ===============================================================
pred_df    = pd.DataFrame(pred_records)
pinball_df = pd.DataFrame(pinball_records)
avg_pinball = (pinball_df.groupby('tau')['pinball_loss']
               .mean().reset_index()
               .rename(columns={'pinball_loss':'avg_pinball_loss'}))

pred_df.to_csv('qrf_v2_tuned_preds.csv', index=False)
pinball_df.to_csv('qrf_v2_tuned_pinball.csv', index=False)
avg_pinball.to_csv('qrf_v2_tuned_avg_pinball.csv', index=False)
print("Saved: qrf_v2_tuned_preds.csv, qrf_v2_tuned_pinball.csv, qrf_v2_tuned_avg_pinball.csv")


KeyboardInterrupt: 

In [ ]:
# === Evaluation harness + reliability figures ===================================
import re, math, matplotlib.pyplot as plt
OUTDIR = Path("results"); OUTDIR.mkdir(exist_ok=True)
FIG_DPI = 140

pred_df = pd.read_csv("qrf_v2_tuned_preds.csv", parse_dates=["timestamp"])
assert {"token","timestamp","y_true"}.issubset(pred_df.columns)

def infer_tau_cols(df):
    tau2col = {}
    for c in df.columns:
        m = re.fullmatch(r"q(\d{1,2})", c)  # q5,q10,...
        if m:
            tau2col[round(int(m.group(1))/100.0, 2)] = c
    expected = set(quantiles)
    if not expected.issubset(tau2col):
        raise ValueError(f"Missing quantiles. Found: {sorted(tau2col)}")
    return dict(sorted(tau2col.items()))
TAU2COL = infer_tau_cols(pred_df)
TAUS = list(TAU2COL.keys())

def wilson_ci(k, n, alpha=0.05):
    if n == 0: return (np.nan, np.nan)
    z = 1.959963984540054
    ph = k/n
    denom = 1 + z*z/n
    centre = (ph + z*z/(2*n)) / denom
    half = (z/denom) * np.sqrt((ph*(1-ph) + z*z/(4*n))/n)
    return (centre - half, centre + half)

# Non-crossing guard (belt-and-braces)
def count_crossings(row, taus=TAUS, tau2col=TAU2COL):
    vals = [row[tau2col[t]] for t in taus]
    return np.sum(np.diff(vals) < -1e-12)
cross_viol = pred_df.apply(count_crossings, axis=1).sum()
if cross_viol>0:
    qcols = [TAU2COL[t] for t in TAUS]
    Q = pred_df[qcols].to_numpy()
    pred_df[qcols] = np.maximum.accumulate(Q, axis=1)

# Pooled metrics
y = pred_df["y_true"].to_numpy()
rows = []
for tau in TAUS:
    q = pred_df[TAU2COL[tau]].to_numpy()
    loss = np.maximum(tau*(y-q), (tau-1)*(y-q))
    q10 = pred_df[TAU2COL[0.10]].to_numpy()
    q90 = pred_df[TAU2COL[0.90]].to_numpy()
    q05 = pred_df[TAU2COL[0.05]].to_numpy()
    q95 = pred_df[TAU2COL[0.95]].to_numpy()
    cover80 = ((y>=q10)&(y<=q90))
    cover90 = ((y>=q05)&(y<=q95))
    c80_lo, c80_hi = wilson_ci(cover80.sum(), cover80.size)
    c90_lo, c90_hi = wilson_ci(cover90.sum(), cover90.size)
    rows.append({
        "tau": tau,
        "pinball_mean": float(loss.mean()),
        "pinball_se": float(loss.std(ddof=1)/math.sqrt(loss.size)),
        "coverage80": float(cover80.mean()),
        "coverage80_lo": float(c80_lo),
        "coverage80_hi": float(c80_hi),
        "width80_mean": float((q90-q10).mean()),
        "coverage90": float(cover90.mean()),
        "coverage90_lo": float(c90_lo),
        "coverage90_hi": float(c90_hi),
        "width90_mean": float((q95-q05).mean())
    })
pooled = pd.DataFrame(rows).sort_values("tau")
pooled.to_csv(OUTDIR/"tbl_metrics_by_tau_qrf.csv", index=False)

# Reliability (global)
rel_rows=[]
for tau in TAUS:
    hits = (y <= pred_df[TAU2COL[tau]].to_numpy())
    ph = hits.mean()
    lo, hi = wilson_ci(hits.sum(), hits.size)
    rel_rows.append({"tau":tau,"hit_rate":float(ph),"lo":float(lo),"hi":float(hi)})
rel_global = pd.DataFrame(rel_rows)
rel_global.to_csv(OUTDIR/"tbl_reliability_global.csv", index=False)

plt.figure(figsize=(5.2,4.2))
plt.plot(rel_global["tau"], rel_global["hit_rate"], marker="o")
plt.plot([min(TAUS), max(TAUS)], [min(TAUS), max(TAUS)], "--")
plt.errorbar(rel_global["tau"], rel_global["hit_rate"],
             yerr=[rel_global["hit_rate"]-rel_global["lo"], rel_global["hi"]-rel_global["hit_rate"]],
             fmt="none", capsize=3)
plt.xlabel("Nominal quantile (τ)"); plt.ylabel("Empirical hit-rate  P(y ≤ q̂τ)")
plt.title("Reliability curve — Global"); plt.tight_layout()
plt.savefig(OUTDIR/"fig_reliability_global.png", dpi=FIG_DPI); plt.close()

# Reliability by regime (uses vol_regime or width-terciles fallback)
df_reg = pred_df.copy()
if "vol_regime" in df.columns:
    # bring regime from original df (merge on token+timestamp)
    df_reg = df_reg.merge(df[["token","timestamp","vol_regime"]], on=["token","timestamp"], how="left")
    df_reg["regime"] = resolve_regime_labels(df_reg)
else:
    width80 = df_reg[TAU2COL[0.90]] - df_reg[TAU2COL[0.10]]
    df_reg["regime"] = pd.qcut(width80, 3, labels=["narrow","mid","wide"]).astype(str)

rel_reg=[]
for rg, g in df_reg.groupby("regime"):
    y_r = g["y_true"].to_numpy()
    for tau in TAUS:
        q_r = g[TAU2COL[tau]].to_numpy()
        hits = (y_r <= q_r)
        ph = hits.mean()
        lo, hi = wilson_ci(hits.sum(), hits.size)
        rel_reg.append({"regime":rg,"tau":tau,"hit_rate":float(ph),"lo":float(lo),"hi":float(hi)})
rel_by_regime = pd.DataFrame(rel_reg)
rel_by_regime.to_csv(OUTDIR/"tbl_reliability_by_regime.csv", index=False)

plt.figure(figsize=(6.2,4.4))
for rg, g in rel_by_regime.groupby("regime"):
    g = g.sort_values("tau")
    plt.plot(g["tau"], g["hit_rate"], marker="o", label=str(rg))
plt.plot([min(TAUS), max(TAUS)], [min(TAUS), max(TAUS)], "--")
plt.xlabel("Nominal quantile (τ)"); plt.ylabel("Empirical hit-rate")
plt.title("Reliability curve — By regime"); plt.legend(frameon=False)
plt.tight_layout(); plt.savefig(OUTDIR/"fig_reliability_by_regime.png", dpi=FIG_DPI); plt.close()

# Coverage vs nominal + width distributions
q05 = pred_df[TAU2COL[0.05]].to_numpy()
q10 = pred_df[TAU2COL[0.10]].to_numpy()
q90 = pred_df[TAU2COL[0.90]].to_numpy()
q95 = pred_df[TAU2COL[0.95]].to_numpy()
cover80 = ((y>=q10)&(y<=q90)); cover90 = ((y>=q05)&(y<=q95))
c80_lo, c80_hi = wilson_ci(cover80.sum(), cover80.size)
c90_lo, c90_hi = wilson_ci(cover90.sum(), cover90.size)
cov_tbl = pd.DataFrame({
    "interval":["80%","90%"],
    "coverage":[float(cover80.mean()), float(cover90.mean())],
    "lo":[float(c80_lo), float(c90_lo)],
    "hi":[float(c80_hi), float(c90_hi)],
    "mean_width":[float((q90-q10).mean()), float((q95-q05).mean())]
})
cov_tbl.to_csv(OUTDIR/"tbl_interval_coverage.csv", index=False)

plt.figure(figsize=(5.2,4.0))
x = np.array([0,1]); ybar = cov_tbl["coverage"].to_numpy()
yerr = np.vstack([ybar - cov_tbl["lo"].to_numpy(), cov_tbl["hi"].to_numpy() - ybar])
plt.errorbar(x, ybar, yerr=yerr, fmt="o", capsize=4)
plt.hlines([0.80, 0.90], xmin=-0.3, xmax=1.3, linestyles=["--","--"])
plt.xticks(x, cov_tbl["interval"]); plt.ylim(0.6, 1.0)
plt.ylabel("Empirical coverage"); plt.title("Interval coverage vs nominal")
plt.tight_layout(); plt.savefig(OUTDIR/"fig_interval_coverage.png", dpi=FIG_DPI); plt.close()

plt.figure(figsize=(5.2,4.0))
plt.boxplot([q90-q10, q95-q05], labels=["80% width","90% width"], showfliers=False)
plt.ylabel("Width"); plt.title("Interval width distributions")
plt.tight_layout(); plt.savefig(OUTDIR/"fig_width_distributions.png", dpi=FIG_DPI); plt.close()

print("Saved tables/figures to:", OUTDIR.resolve())


## **What I did.**
I rebuilt the QRF rolling pipeline on Zillow’s `quantile-forest` with (i) regime-aware residual quantile offsets $\delta_\tau=Q_\tau(r)$ computed on a NaN-filtered calibration window, (ii) isotonic non-crossing, and (iii) split-conformal inflation for the 80%/90% two-sided bands. I hardened the code against NaNs from warm-up or imputation and ensured the `vol_regime` quintiles map correctly to `"quiet"`, `"mid"`, `"volatile"`.

## **Why.**
This preserves good per-τ reliability while pushing two-sided coverage toward its nominal targets, and keeps tuning/evaluation stable across folds and tokens.
